In [2]:
import pandas as pd
import numpy as np

In [3]:
train = pd.read_csv("../data/WN18/train.txt",delimiter="\t", header=None)
test = pd.read_csv("../data/WN18/test.txt",delimiter="\t", header=None)
validation = pd.read_csv("../data/WN18/valid.txt",delimiter="\t", header=None)
train.columns = ['Header', 'Tail', 'Relation']
test.columns = ['Header', 'Tail', 'Relation']
validation.columns = ['Header', 'Tail', 'Relation']

In [4]:
entities = pd.concat([train.Header, train.Tail])
entities = pd.DataFrame(entities,columns=['ID'])
entities = entities.groupby('ID').ID.agg(['count']).reset_index()


In [5]:
relations = pd.DataFrame([], columns=['ID'])
relations['ID'] = train.Relation
relations = relations.groupby('ID').ID.agg(['count']).reset_index()

In [9]:
relations

,ID,count
0,_also_see,1299
1,_derivationally_related_form,29715
2,_has_part,4816
3,_hypernym,34796
4,_hyponym,34832
5,_instance_hypernym,2921
6,_instance_hyponym,2935
7,_member_holonym,7382
8,_member_meronym,7402
9,_member_of_domain_region,923


In [5]:
test['Head_idx'] = test.index.map(mapper=(lambda x : entities.index[entities.ID == test.Header[x]].tolist()[0]))
test['Tail_idx'] = test.index.map(mapper=(lambda x : entities.index[entities.ID == test.Tail[x]].tolist()[0]))

train['Head_idx'] = train.index.map(mapper=(lambda x : entities.index[entities.ID == train.Header[x]].tolist()[0]))
train['Tail_idx'] = train.index.map(mapper=(lambda x : entities.index[entities.ID == train.Tail[x]].tolist()[0]))

validation['Head_idx'] = validation.index.map(mapper=(lambda x : entities.index[entities.ID == validation.Header[x]].tolist()[0]))
validation['Tail_idx'] = validation.index.map(mapper=(lambda x : entities.index[entities.ID == validation.Tail[x]].tolist()[0]))

train['Relation_idx'] = train.index.map(mapper=(lambda x : relations.index[relations.ID == train.Relation[x]].tolist()[0]))
test['Relation_idx'] = test.index.map(mapper=(lambda x : relations.index[relations.ID == test.Relation[x]].tolist()[0]))
validation['Relation_idx'] = validation.index.map(mapper=(lambda x : relations.index[relations.ID == validation.Relation[x]].tolist()[0]))

train = train.drop(['Header', 'Tail', 'Relation'], axis=1)
test = test.drop(['Header', 'Tail', 'Relation'], axis=1)
validation = validation.drop(['Header', 'Tail', 'Relation'], axis=1)

train = train.to_numpy()
test = test.to_numpy()
validation = validation.to_numpy()

In [6]:
totalWords = sum([freq[1] **(3/4) for freq in entities.values])     
entities['Probability'] = (entities['count']**(3/4) / totalWords)

totalWords = sum([freq[1] **(3/4) for freq in relations.values])     
relations['Probability'] = (relations['count']**(3/4) / totalWords)

In [7]:
entities.reset_index(level=0, inplace=True)
entities = entities.drop(['ID'], axis=1)
entities = entities.to_numpy()

relations.reset_index(level=0, inplace=True)
relations = relations.drop(['ID'], axis=1)
relations = relations.to_numpy()

In [9]:
np.save('../data/WN18_numpy/train.npy', train)
np.save('../data/WN18_numpy/validation.npy', validation)
np.save('../data/WN18_numpy/test.npy', test)

np.save('../data/WN18_numpy/entities.npy', entities)
np.save('../data/WN18_numpy/relations.npy', relations)